# WebSocket API MRN Example with Python

## Prerequisite

This article/notebook is focusing on the Refinitiv Machine Readable News (MRN) data processing only. I highly recommend you check the  [WebSocket API Tutorials](https://developers.refinitiv.com/en/api-catalog/elektron/refinitiv-websocket-api/tutorials) page if you are not familiar with WebSocket API. 

The Tutorials page provide a step by step guide (connect, login, request data, parse data, etc) for developers who interested in developing a WebSocket application to consume real-time data from Refinitiv Real-Time. 


## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

MRN aims for replacing the legacy News 2K (N2_UBMS and N2_STORY).

### MRN Data behavior

MRN is published over Refinitiv Real-Time using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (RTSDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/mrn_process.png"/>

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Enterprise Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Refinitiv Real-Time Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [2]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib

In [3]:
# ADS connection variables

hostname = '127.0.0.0'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [4]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_news_messages = []

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

 

In [5]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_STORY'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [6]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [7]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [8]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [9]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(30)
    web_socket_app.close()

Connecting to WebSocket ws://172.20.33.30:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"10.42.85.159"
    },
    "Name":"root"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":61430,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"10.42.85.159",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":1,
        "SupportPauseResume":1,
        "SupportStandby":1,
        "SupportViewRequests":1
      },
      "Name":"root"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Open",
      "Text":"Login accepted by h

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/41V224bNxB971cM9qFoAWVly1LcLBAUukV2LEuOLk2TqiioXUq7EZfc8GJZLQr0N/qWb8mn9Et6uJQV56FAYXs5pGeGM2cOZ/6ImLDXWZRE8m40fPN2Nx+zqBExlxVcptxEyS/R5C7pz14NR9GvjWitsgOUXylNa24saW6csIb2OZdU6ULaQm7J5oUhJqVy8FFyaRtUCc4Mp1QU6Y6UJFHIHVwItU9WMre2SprNKtvEmjvLtYlTVTZzWwrJ9+YkxMxUPxYv2xdnnRfpxXpzxi8uO+3zb91Lp2XiNUqRPPGQtM5aZ2et1vPkS3Yr6X8WOaflDZmDsbwkVXHJMzJWK7kVB3IygwdXVaLAsVUZO/iYLYzgdV1IZgvs1WYlgw3XlPGSyYz8H5LCQW1/IKnKo76JaXLa0AYQjicjMlxmytmVzLSqqvo66jwv07KZ+csqpsPZD61wdgDsXCOimAawoELST/WRgB4ri2wF2C3bcmKWbpQQRnJTh7W86c9RI5W5NIRfqxmvN7cxveJ66wxN5rd3tfqCc2OKjK9kv7sw31Hfid85k98TIqKirJS2TFpaOyEKk9OGpVZpU+f1dhDTFUC457pBe078oeKpPeILzJxGeAj8Sf5IsVRaqz2oVKQ5mVw5kdHnTwj4HlEQA1uY9jeZYiuZqC8adGNfzXntmPaFzAyZivPMHN3k7J4jZ0QiLfBJmd4qpLzmGhyFCXK/LQRcZXQFVfgSDLTec76r8wRpvZpEXDHNfUK4+dGLVwCP4ddVHk2PFs0h5Mhe7TyKKznQbAuw4c

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/41XXXPbuBV976/AaOr2RdJKTOxs9FTHkRPFK9kbeevslp0ORF6RsEiAC4CWtZ3+954LUJQ6eanHI35d4H6de+7Fvwey8ot8MBvoFel3q+z57XowHMg2V6QzcoPZPwarh9lqvhr8czjYmPwA0VR/OIib0uhC3JlSPCjcSJ2La62lWDgpM5FqwX+fKrORlXDeZDsnMqO90i3lQuK+tY6E2eIjUSMcVZXZbp14olyTy+VByLAiI6udMC9khS8p1YR3plYZvjnP6/FWaNrj2RotX5RtnSgk3looamRG46M1X6lQEKmE0jm9khNbKBUys8Y5MW+taSj64ZQci0fs20g9ilaT9li49ub19Sh6NZmkOsdtA0XJeHoxFHvlS/GJbC314a9OfLz+FnWJimTOYaqgCoqDnBRbCf1wIRknF2PxweAle4NlqWZDvkgYgH1WarcjJZLkEipKlZXCldA5Gf8InZZkxk9YqSwU7AlxqeiFKiecQvzEfebNhuw41det81ZWSmLT9V8efrhefxPJZCJOXry5CCG4M9gWQnfGNQoqPB49BKYQ6OP5y3g9FtvWtxYuecU2mOiA2YsvBmkUC52zSoXYXSOFsiCxR2JEbvYaWZO2OojphQg3qe5zH6PP24QUdYLvYWljlPbAgxaPbYQJdBIsll4ofDj3P7qf6s5/Ibcev8HqkoD7ErHfqgzGIVJShRAeBL02lHlkZ69yiLsGrudHoJ2BLKZIwhlINNJyercGWmCjB1zYZ6QvJ4C1Dxm7dVBU5ewBbxismU5GB7goHqHKtRY+WZ

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/61UwVLjRhC95yu6dIKK8RqxIYVPCd6wJmB7d42LBNhKtaSWNCDPeGdGNmIr/543kg3ZhEMOKVmyetTTr/u96f4aceXPs2gY6fOz355uTsaPHPUirjMlOhUXDW+j6YchvkWfe1Fisgaud3oxpZUVJ9o7yiznnriwIksskNF0qZKGyXlOVKWe2CuseUOfaucU0wGdmiJjbdZ3GpefzOaj2XWfziSh+Jj2zrUXm/PjPjznK0kVV/RJtngIthZindFYOCOTky+FFlp5yWjaQjma16uVsZ4mCoDAVruU9hbT+eT8cp/el+wca5pzxUshWF0dCMKvpv9SX8DeJgNv5V8q623/Nb2TVe0bOjNWVKGRh1YOReHloWRVPRNAjlXW72i4i15JKuWqAgosJ2uxYMKrpTgyMMhKGhLKuHFtVili27aC1OhcbFAw1P5etKyZNsqXsAJfuQXAxtiHHYE/0Pe4fzVKB9Yq5dk2NDLLpfJepEebUqUlYq1NtQaC6wgW60IArg7mbC3f0235N1FaGoEectXd0ci3EqGOn9PU2CzUYAqtnhA0aTotp3TGjTy9hP0czpTRgRFUGe5v8ccQji3dbgFfIOyyKwJbQNdFyZXKeeveBg14Bg/bhu7dRd8qQ+xbl52sH+qkUimNSl4m2IMA15JpcVDgWcRxJxz0p9zYDdttxqH+2rXhvDz6HUmvdM8/Dl7Y7KFlAXetdLHbmQo7ySF4jxKApWI9Q96VNUklSwdml7D/VVOXKK4rIXnYGXuz2pJkyhsb2g0p5DkOn6ZEEIjTUloykE

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/7VUz2/jRBS+81c8+YBAm7qOnSbNSCuUJm1T2oZsnO0upQiN7Uk9W2fGOx4n7SIO/CgrCtqKCwdWQlohJA5IewIK++802/4ZvLGTgnYFN+LI75uZN997/ubN+9iiid6KLGKJ7rZf84J3Q2FVLJpHnImQZRb5wOr1SXd7y7c+rFiBjE7Qd3b+7fWzb66+/2J2/vz61yezpz9cffX06suz6+efXr74cXb++4q34ngVMKZ++cfXL5/8PHv85+z0l9nF57Pzs+tnP81OTy8vzmYXn7387vHli9/emnIdw93BzttwIBbP+nEYU3HIYFiFkGp2KNUJhDJi0HDwRw6sDgs0UBGBr1Ue6lyxCPpKRoizA+tVFvdVFrdesOzQXIQxtISQCNiYCQ1L0F5ba5cc/8HglQx+nqZJsZEmsMMzzcUhdGSYv8Z1IGKtU7K8nEYjW7FcM5XZoRybsWDTbGFtmqXv8Ns1z1lphl4wcpjXWKlV38xv50oQ4zFOyD/2E9dxHcd168ScY3Pgr6oNE62QsyPzIGEQJjw8AilAx8xoDTSQEwZawoSzaTFLleZhwmzoJ4xmDITU6BBTDVxgKMFQ7hDLIgOegWIPc46S27A1ghOZAztOmSrqBlIlMeR47m3kMPQLkgqkJX8oRZYn2uxWgAtTqY6ARmMuUERFtVSAf83CWPAQtc1QaKn0zZft4HFkGiaoAscPkyNoRTJg0AoxPmatGI2YKpNFlfA4IiyRxRf3Oxsw4gnLbIAhppe8xmaSnnOEVAAyy0BTLpBkpOQY5oc5nU5tagKXRz

...

RECEIVED: 
[
  {
    "Type":"Ping"
  }
]
SENT:
{
  "Type":"Pong"
}
RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/41X33PquBV+71+hYZp2dy6w4GxyF55KiENIAqGY3KRbOh1hC6yLkbiSDGE6/d/7HckQMu1DX6xj++j8Pt+R/lXjhRtmtW5N3T5MO5erb7NVrV7jZSaFSoWtdf9eG0+6t/EYX4l46J2o/olIjtTdkRiMjtSs9o96baGzA3TM1VzdHFg/12rFHnXOJhIEVxnrKcXZ0HKeMmIaFHrBC2adTteWpVo5qUqRMQ66NFYwvcRPIbbMiqLQy6VlryJTwmb8wLjfkQqjLNM7YebK5YIJfNMbmeKfdbSfPiqxx7vRiu+kKS1bcXw1ULTlqWiSJVOxkvhdMKky8S4sW0Ih46nR1rK4NHorggNW8iabQeaWq0awWCiHjYnT7+8V61xdt1osA7mFkqjZvqizvXQ5Gwiz4erwZ8tue29BFysEzyg+BVRBsefjbMmhH+ZHzegCFt5ofCVXaB8Z8sBhAOSM5XotJIuiK6jIZZozm0Nnq/kbdBrBU3rDRmmgYC+sm6tC7ERhmZUIHntOnV4I02S90jrDC8khNPnT5Jde8saiT15cXnjNjxpiwfSo7VbOlREO7w4cbXD4YL40kyZblq408MdJMkAH4/WePWjkjw1VRvokAtdD7vhKsD0ywjK9V0gXN8WBtS/mFXVKegg9ifH5qX53YOZWS+VQCIrNylAf0ClgLndMOguvvffMO//Zd8aXDk9vdS7QKDkCv5QpjEOUuPTxOzDxvhW

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/41Wy3LbNhTd9yswXNMqRduKpZ1sS46TSHYkP6auOxlQhERYJMjiYcXq9De6zcbfkJV3+rGeC9KK7XTReCKAeFyce8+5F/gr4Lk9TYNeoPoVj9r7bR6EAXepFGomTND7PRif9/rn/WHwRxgkZfqApZc2b/XYWFhbFlJYofPNdyNYLuQss2wh5jzPhWI77Fjey1SoVLD55kmzOGp3mVMpOnF0q5j/lwjJojB+xwZOl7eK/o65YdzN2ZmxAoMVZ6YSa8lzaaTQVrBk810boVRp6+/Toih3LhWQKJEVOLpvtXQFmwies4GxHGsybpks2Ikws2zzOLfmjmcNJCOkwibv0EKspFIsFc7mcAfeAIRc4JgWOxYa+/284pibAh6OZAYwFiznzrL7EsF4dHPsUOyGZxQHANnthrvsXJdroax37WAv3GMjWba82yFDvFkKt186wQussHZV4qgCHSFzK1o+QFj9U/jnUjSntWH75WHtd52Xp7XYWaYEmzhjcg42PHzsNB7DwvcbLmEsCg9eG9uLwu5LY0OilrB/qAMaR6yCXftMAgX3hxIUx7Fg4MlapxaIl/rBPrsTrL8EmtDbW20etQABtKVk93CPrQCRKPNRaOwnwki7tnX8iSrisq8WPBHEo14LaVnc8SJJZA5hUzQ9LM0OtWvCOAdvs0x4RJunBHMHUdSKooh9djzV3BYUauX1S3u9lHhhV5CG36S9ssMOG+n0OTbHbyiVBiJU7LwEUyGBYxcIhphlkhoFwVYuyeWynpvm5YovkR9keUuX3TxauQjZqqTUoU

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/7VSTW/bMAy971cQwg4tENeyHDuNMAxI4qwrmjhO7AL76BDIsZJ6ce1Ekr0Gw/776Lo57LRddhEfKYriI99PIgpzmxFOysj0/eZ4JD0i6iyX5UZqwr+SMOLRzZh865G0yk6YyPwPMmXUBjp0KIzmMA0S+wXfzJOH0rLOmENUCGM0jOtyLxW41xQ22sCFR6nlU3oJQdFkEFe1eYS7SkkBF/eFFqU9rvG8hLlQzMKj34PFZBGG00kC76L3AErkWmpI8wzeup575VHYVgqwJgy5QzkdXPmec+7nzy5GhVTGsh5KJLrNlTYT/NjIdgSMMmpRZjE/oec61wPvC2Y+SpEVeSkxK5qNkiTmMBwOLCuUP+BzpfYw8P8b3b+xxfbytv3XBa7Xa0YpYz6Ln2dlvTXClf2mDsyn7T6KjifP7duNHX6MqmZ5s1y2r0ttRLvvxRY3jpsuRLmrxa5lK9s5PUmt0U1OBwwx9POnV4cY+WzsQyHyNu+gqibPpMJ4GPNolsRtsE5jI0yNaiL4j+HINi0k3ug6/S435kVmU+5gZMr7tDN9NHPuLAO0K1TgiOK8z7iDIePj0STp0CRaLYIO3obBfYdm07ADi1WwaiVsxF7G8li36ibc6ZFa7RCjrN0eaaTSeVX+gx5+vfkNqmM1EDgDAAA=",
      "FRAG_NUM":1,
      "GUID":"Pt46vqq___2002262SxLnufta3e4vuDtXfkPPqy534/v/NHPovQGQQ",
      "MRN_SRC":"HK1_PRD_A",
      "MRN_

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-26",
      "FRAGMENT":"H4sIAAAAAAAC/41W23LbRhJ9z1d08UmKSQiASJFEbW2FIWlJZYmSSWodZ51SgcCQnBCcgTEDMczWVu37fom/w3+yX7KnZ6iL7WxtXjCNufTldPeZ+UcjLexl3kga6vxu2p/HH81Do9lI61wKlQnTSP7emNwmWGv80mwsdL7H1g9qWMhsQ2tRCbKajBCk64qydVrZhD6oD2ptbWmSkxM3ZQJbpblUq0woSEWQ6e1h5SQX21TZfLOJwzCOz6IwPIvOgpVcspYbKC3lg7ZUaqksGZuq3FBqKY56QTt42lOJJXyBwwnZtaBc75SRueCFh1QWOGKo0GrFoztK0lAljGSNmXCKBoUVlUqtfBBk4GlaSZ1QutD4d2eaUKE3tNQVxXE36BKcofj0NIjd+aHebhGf92A6u2QTC1HoHXXCgOaYvB4MRzyrxMqbYQV+i7SGjFyptKBCKuH3Z1ot5aqusFmrlwcDutaVgGNV01kzwFI8ovOsMA6dhU5IebqHdTpCyKXIWEWxdzDGnSA684FwiO3jgF5LeFHsmzRyqWG7h/S9UI1RVFAroedHXcDnFf4XrOjI+cHb2UDYD3rHDp9ZXZYaKXfWnrE3CS86GOn7753swPWiy9VhOmoHZ04I29h7NHrzplWkxh7zXNQ/DdpO6PWC6HAi6sVBzyuKut0nRa1Wi4f5dDC6nJzTcDyBeMVxpoDcJTGt9qjq6kFmgBWIybSQhkOSiqzI1kpmyFQKoPaIw4X3GmVh0kIYEupjLSsJqSxEapBH4El77pCpqFFkMJRluga2ADhl4HD2yR3fI6xysa

News = {'altId': 'nBQN2MGjQ', 'audiences': ['NP:BEQN'], 'body': 'For best results when printing this announcement, please click on link below:\nhttp://pdf.reuters.com/htmlnews/htmlnews.asp?i=43059c3bf0e37541&u=urn:newsml:reuters.com:20200226:nBQN2MGjQ\n\n============\n\n   REAL HOLDINGS FÖRETRÄDESEMISSION TÄCKT AV VILLKORADE TECKNINGSFÖRBINDELSER\n   TILL CA 92,6 PROCENT\n\n   Den 30 januari 2020 kallade styrelsen i Real Holding i Sverige AB (publ)\n   ("Real Holding" eller "Bolaget") till extra bolagsstämma att hållas den 2\n   mars 2020 för att bland annat besluta om en 35 MSEK företrädesemission.\n   Styrelsen kan idag meddela att emissionen till ca 92,6 procent täcks genom\n   villkorade teckningsförbindelser om ca 32,4 MSEK.\n\n   Teckningsförbindelserna avser teckning både med och utan företräde i\n   emissionen. Teckningsförbindelserna utan företräde kan enbart bli aktuella\n   att ta i anspråk om emissionen inte blir fulltecknad av Bolagets\n   befintliga aktieägare, vilka har 

### Real-time News Data Model
The news data appears as JSON in UTF-8 after decompression and assembly of the individual messages.
The Real-time News feed contains the headline, story body text, and associated metadata about the story as a simple group of named values.

Example JSON fields are following:
- ```firstCreated```: UTC timestamp for the first version of the story. Millisecond precision.
- ```headline```: The headline text of the news item.
- ```id```: Uniquely identifies the news item. This is the same value as the GUID in the OMM envelope.
- ```body```: The full body text of the news item (Story Body).

For more detail regarding MRN data model please visit [MRN Data Models and Refinitiv Real-Time Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item) page. If you want to compare MRN and the legacy N2_UBMS news data model, please visit [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).



In [10]:
print("first 10 headlines\n")
for news in _news_messages[:10]:
    if news["headline"]:
        print(news["headline"])

first 10 headlines

GETBACK SPÓŁKA AKCYJNA W RESTRUKTURYZACJI - Uprawomocnienie postanowienia Sądu Rejonowego dla Wrocławia-Fabrycznej we Wrocławiu Wydział VIII Gospodarczy do spraw Upadłościowych i Restrukturyzacyjnych z dnia 6 czerwca 2019 r. o zatwierdzeniu Układu
***Volatilitätsindex VSMI +18% auf höchsten Stand seit Anfang 2019
CRF: European Gas  NBP DA Outlook - Bearish
Monex Europe Limited - Markets convulse as coronavirus outbreak teeters on the brink of becoming a pandemic
NIKKEI Coronavirus Concerns Fuel Fresh Selloff in Global Stocks
Reuters Insider - Fiat Chrysler deal is 'not a merger in crisis mode,' Groupe PSA chairman says
UN presents draft agreement on Libya stabilization to Russia - Bogdanov
Reuters Insider - Coronavirus will 'certainly' be worse than SARS epidemic, analyst says
發行的可贖回牛熊證代號53503, 53506之推出公佈及補充上市文件(with URL)
发行的可赎回牛熊证代号53503, 53506之推出公布及补充上市文件(with URL)


In [11]:
print("first 10 stories\n")
for news in _news_messages[:10]:
    if news["body"]:
        print(news["body"])

first 10 stories

 KOMISJA NADZORU FINANSOWEGO  

 

  Raport bieżący nr 8 / 2020  

  

 Data sporządzenia:  2020-02-26          

 Skrócona nazwa emitenta          

 GETBACK S.A.  

 Temat           

 Uprawomocnienie postanowienia Sądu Rejonowego dla Wrocławia-Fabrycznej we Wrocławiu Wydział

VIII Gospodarczy do spraw Upadłościowych i Restrukturyzacyjnych z dnia 6 czerwca 2019 r. o

zatwierdzeniu Układu  

 Podstawa prawna         

 Art. 17 ust. 1 MAR - informacje poufne.

  

 Treść raportu:          

 Zarząd GetBack S.A. w restrukturyzacji (Emitent) informuje, iż powziął informację o wydaniu w

dniu 24 lutego 2020 r. przez Sąd Rejonowy dla Wrocławia-Fabrycznej we Wrocławiu w sprawie o sygn.

akt VIII Gz 1/20, VIII GRp 4/18 postanowienia o oddaleniu zażalenia wierzyciela (Postanowienie) na

postanowienie z dnia 30 grudnia 2019 r. o odrzuceniu zażalenia wierzyciela na postanowienie o

zatwierdzeniu układu z dnia 6 czerwca 2019 r., o którym Emitent informował raportem bieżącym nr


In [12]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install pandas and numpy packages in a current Jupyter kernal\n

# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install numpy

In [13]:
import pandas as pd

In [14]:
df_headlines = pd.DataFrame(_news_messages, columns = ["firstCreated","messageType","provider","language","headline"])

In [15]:
messageType = {
    0: "Unknow",
    1: "Alert",
    2: "First take",
    3: "Subsequent take",
    4: "Correction",
    5: "Corrected",
    6: "Update",
    7: "Deletion",
    8: "Drop due to expiry"
}

In [16]:
df_headlines["messageType"] = [messageType[value] for value in df_headlines["messageType"] if value in messageType ]

In [17]:
df_headlines

,firstCreated,messageType,provider,language,headline
0,2020-02-26T09:09:42.404Z,First take,NS:EMIT,pl,GETBACK SPÓŁKA AKCYJNA W RESTRUKTURYZACJI - Up...
1,2020-02-26T09:09:44.115Z,First take,NS:AWP,de,***Volatilitätsindex VSMI +18% auf höchsten St...
2,2020-02-26T09:09:44.457Z,Corrected,NS:CRF,en,CRF: European Gas NBP DA Outlook - Bearish
3,2020-02-26T09:09:44.000Z,First take,NS:PUBT,en,Monex Europe Limited - Markets convulse as cor...
4,2020-02-26T09:09:45.807Z,First take,NS:NEN,en,NIKKEI Coronavirus Concerns Fuel Fresh Selloff...
5,2020-02-26T09:09:02.091Z,Corrected,NS:CNBC,en,Reuters Insider - Fiat Chrysler deal is 'not a...
6,2020-02-26T09:09:46.107Z,First take,NS:IFX,en,UN presents draft agreement on Libya stabiliza...
7,2020-02-26T09:09:46.221Z,First take,NS:CNBC,en,Reuters Insider - Coronavirus will 'certainly'...
8,2020-02-26T09:07:15.978Z,Update,NS:HIIS,zh,"發行的可贖回牛熊證代號53503, 53506之推出公佈及補充上市文件(with URL)"
9,2020-02-26T09:07:15.994Z,Update,NS:HIIS,zh,"发行的可赎回牛熊证代号53503, 53506之推出公布及补充上市文件(with URL)"


In [18]:
df_story = pd.DataFrame(_news_messages, columns = ["headline","body"])

In [19]:
df_story

,headline,body
0,GETBACK SPÓŁKA AKCYJNA W RESTRUKTURYZACJI - Up...,KOMISJA NADZORU FINANSOWEGO \n\n \n\n Rapor...
1,***Volatilitätsindex VSMI +18% auf höchsten St...,\n\n
2,CRF: European Gas NBP DA Outlook - Bearish,For best results when printing this announceme...
3,Monex Europe Limited - Markets convulse as cor...,\nGBP\n\nSterling managed to find its footing ...
4,NIKKEI Coronavirus Concerns Fuel Fresh Selloff...,\nBy Chong Koh Ping and Anna Isaac \n Glob...
5,Reuters Insider - Fiat Chrysler deal is 'not a...,\nClick the following link to watch video: htt...
6,UN presents draft agreement on Libya stabiliza...,\nUN presents draft agreement on Libya stabili...
7,Reuters Insider - Coronavirus will 'certainly'...,\nClick the following link to watch video: htt...
8,"發行的可贖回牛熊證代號53503, 53506之推出公佈及補充上市文件(with URL)","發行的可贖回牛熊證代號53503, 53506之推出公佈及補充上市文件(with URL) ..."
9,"发行的可赎回牛熊证代号53503, 53506之推出公布及补充上市文件(with URL)","发行的可赎回牛熊证代号53503, 53506之推出公布及补充上市文件(with URL) ..."


## References

* [Refinitiv Real-Time SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.refinitiv.com/) web site.
* [Refinitiv WebSocket API page](https://developers.refinitiv.com/en/api-catalog/elektron/refinitiv-websocket-api).
* [Developer Webinar Recording: Introduction to WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Enterprise Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Refinitiv Real-Time Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).